In [1]:
import numpy as np
import cvxpy as cp
import json 
import json 
import pandas as pd 
import polars as pl 
import os 
from lower_mpc_old import step_lower_mpc, LMPCDataBuffer
from upper_mpc import step_upper_level, UMPCDataBuffer
from tqdm import tqdm 
import time 
from datetime import timedelta
import matplotlib.pyplot as plt 


%load_ext autoreload
%autoreload 2

In [2]:
root_folder = "/home/alqua/papers/mbs_aggregation_paper_code/mpc/"
params_path = "sys_id/results/"

In [3]:
data_path_wsl = "/home/alqua/data/pump_station_data/"
upper_mpc_data = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
upper_mpc_data_pan = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))

In [4]:
start_date = pd.to_datetime("2024-02-15 00:00:00+00:00")
end_date = pd.to_datetime("2024-02-25 08:00:00+00:00")

upper_mpc_data = upper_mpc_data.filter((pl.col("time") >= start_date) & 
                                       (pl.col("time") <= end_date)
                                       ).with_columns(pl.col("CO2Emission")/1000, pl.col("price")/1000)

upper_mpc_data = upper_mpc_data.with_columns(pl.col("time").dt.day().alias("day_index"))
day_sliced_df = upper_mpc_data.filter(pl.col("day_index") == 15)

In [5]:
with open('lower_mpc_coefficients.json', 'r') as json_file:
    models_coefficients = json.load(json_file)

In [22]:
lmpc_data = LMPCDataBuffer()
umpc_data = UMPCDataBuffer()
lmpc_data.initialize()

slice_df = upper_mpc_data.select(pl.col(["time", "inflow_kf", "CO2Emission","price"]))
inflow_kf = slice_df["time", "inflow_kf"].upsample(every="1m", time_column= "time").fill_null(strategy="forward")["inflow_kf"]

k = 0 
zs = 3 
N = 60
horizon = 24
step_size = 1
max_start = len(slice_df) - horizon + 1
start_index = 0 

with tqdm(total=max_start, desc="Hour Steps") as pbar:
    start_index = 0
    while start_index < max_start:
        prices_values = slice_df["price"][start_index : start_index + horizon]
        co2_values = slice_df["CO2Emission"][start_index : start_index + horizon]
        inflow_values = slice_df["inflow_kf"][start_index : start_index + horizon]
        
        start_time_umpc =  time.time()
        ############################################### Upper Level Optimization
        
        umpc_opt_results = step_upper_level(
            horizon=horizon,
            prices_values=prices_values,
            co2_progn_values=co2_values,
            inflow_values=inflow_values,
            h_init=lmpc_data.data["height_sys"][-1],
            energy_init=lmpc_data.data["p1_power"][-1] + lmpc_data.data["p3_power"][-1] + lmpc_data.data["p4_power"][-1],
            Qout_init=lmpc_data.data["qout"][-1])

        end_time_umpc = time.time() - start_time_umpc
        umpc_opt_results["opt_time_umpc"] = end_time_umpc
        umpc_data.update(umpc_opt_results)

        ############################################### Lower Level Optimization

        for k in range(0, 60):
            u_stack = np.vstack([lmpc_data.data["u1"][-zs:],
                                lmpc_data.data["u2"][-zs:], 
                                lmpc_data.data["u3"][-zs:]])

            power_stack = np.vstack([lmpc_data.data["p1_power"][-zs:],
                                    lmpc_data.data["p3_power"][-zs:], 
                                    lmpc_data.data["p4_power"][-zs:]])
            
            start_time_lmpc = time.time()

            start_time_lmpc =  time.time()
            res_dict  = step_lower_mpc(Qin_est = inflow_kf[start_index + k : start_index + k + 63], # da cambiare
                                    Qout_meas = lmpc_data.data["qout"][-zs:],
                                    h_meas = lmpc_data.data["height_sys"][-zs:],
                                    w_meas = u_stack,
                                    E_meas = power_stack,
                                    P_meas = lmpc_data.data["pressure_sys"][-zs:],
                                    h_ref = umpc_opt_results["height"],
                                    trigger = [1,0,0],
                                    N = 60,
                                    zs = zs,
                                    models_coefficients = models_coefficients)
            end_time_lmpc = time.time() - start_time_lmpc
            res_dict["opt_time_lmpc"] = end_time_lmpc

            res_dict['time_utc'] = slice_df["time"][start_index] + timedelta(minutes=k)
            res_dict['height_ref'] = umpc_opt_results["height"]
            res_dict['qin'] = inflow_kf[start_index + k : start_index + k + 1][0] # access value form the list           
            lmpc_data.update(res_dict)

        start_index += step_size
        pbar.update(step_size)
        if start_index == 1:
            break
    opt_lmpc = lmpc_data.to_dataframe(save=True, file_path='save_test_lower.par')
    opt_umpc = umpc_data.to_dataframe(save=True, file_path='save_test_upper.par')


Hour Steps:   8%|▊         | 1/13 [00:35<07:08, 35.73s/it]


In [23]:
opt_lmpc

time_utc,u1,u2,u3,p1_power,p3_power,p4_power,qout,qin,height_sys,height_ref,pressure_sys,opt_time_lmpc
"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-02-15 00:00:00 UTC,816.0,3.0,5.0,33.2,-0.0,0.0,0.0,644.241879,111.0,199.999998,0.592,0.508961
2024-02-15 00:01:00 UTC,831.0,5.0,9.0,35.9,0.1,0.1,496.2,644.241879,151.3,199.999998,0.54,0.567051
2024-02-15 00:02:00 UTC,846.0,7.0,13.0,38.1,0.2,0.3,616.3,644.241879,160.6,199.999998,0.592,0.586857
2024-02-15 00:03:00 UTC,860.0,9.0,16.0,39.3,0.3,0.4,622.0,644.241879,162.3,199.999998,0.607,0.685149
2024-02-15 00:04:00 UTC,874.0,10.0,19.0,40.2,0.4,0.6,622.9,644.241879,163.7,199.999998,0.607,0.676045
…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-15 00:55:00 UTC,1243.0,0.0,0.0,58.5,0.0,0.0,641.5,644.241879,193.3,199.999998,0.61,0.584228
2024-02-15 00:56:00 UTC,1245.0,0.0,0.0,58.6,-0.0,0.0,638.9,644.241879,193.5,199.999998,0.61,0.590269
2024-02-15 00:57:00 UTC,1247.0,0.0,0.0,58.7,0.0,0.0,633.5,644.241879,193.8,199.999998,0.609,0.566802
